In [115]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz


In [2]:
!rm *.hdf5

rm: cannot remove '*.hdf5': No such file or directory


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
fname_dir="/content/blood/"
fname_url="/content/drive/MyDrive/blood/Update_train.csv"
fname=fname_url.split("/")[-1]

In [5]:
fname_dir="/content/blood/"
!mkdir $fname_dir

mkdir: cannot create directory ‘/content/blood/’: File exists


In [6]:
!cp $fname_url $fname_dir

# Download full data set


In [7]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


In [8]:
file_name=fname_dir+fname

In [9]:
import pandas as pd

In [10]:

df= pd.read_csv(file_name)
df.head()

absorbance0  absorbance1  ...  cholesterol_ldl_human  donation_id
0     0.520883     0.528200  ...                   high         2250
1     0.529109     0.534852  ...                   high         2250
2     0.528434     0.532036  ...                   high         2250
3     0.530528     0.531880  ...                   high         2250
4     0.527530     0.536424  ...                   high         2250

[5 rows x 178 columns]

In [11]:
df.describe()
df=df[df.donation_id!=6824]
df=df[df.donation_id!=7148]
df=df[df.donation_id!=6362]
df=df[df.donation_id!=4122]



In [12]:
df.describe()

absorbance0   absorbance1  ...      humidity   donation_id
count  28920.000000  28920.000000  ...  28920.000000  28920.000000
mean       0.508651      0.510847  ...     35.333558   6652.136929
std        0.037351      0.037289  ...      8.654283   3046.276623
min        0.398490      0.399354  ...     16.610000   1975.000000
25%        0.486728      0.488631  ...     28.377500   3592.000000
50%        0.509548      0.511864  ...     35.330000   6428.500000
75%        0.530501      0.532439  ...     41.190000   9351.000000
max        0.657782      0.652291  ...     67.820000  11906.000000

[8 rows x 175 columns]

In [59]:
df["data"]=df["hdl_cholesterol_human"]   # +"+"+  df["hemoglobin(hgb)_human"]+"+"+df["cholesterol_ldl_human"]

In [60]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

_3_Label_encoder=LabelEncoder()
_3_Label_encoder.fit(df["data"])
_3_Label_encoder.classes_
y=_3_Label_encoder.transform(df["data"])
df["y"]=y

In [61]:
# x and y given as array_like objects
import plotly.express as px
fig = px.scatter(df,x="temperature", y="absorbance99",color="y", hover_name="donation_id")
fig.show()

In [62]:
func=["mean"]

In [63]:
df.columns

Index(['absorbance0', 'absorbance1', 'absorbance2', 'absorbance3',
       'absorbance4', 'absorbance5', 'absorbance6', 'absorbance7',
       'absorbance8', 'absorbance9',
       ...
       'id', 'std', 'temperature', 'humidity', 'hdl_cholesterol_human',
       'hemoglobin(hgb)_human', 'cholesterol_ldl_human', 'donation_id', 'data',
       'y'],
      dtype='object', length=180)

In [64]:
cols=list(df.columns[:170])
cols.append("y")
df_agg=pd.DataFrame()
for col in cols:
   
    result=df.groupby("donation_id").agg({col:func})
   
    df_agg[col]=result[col].iloc[:,0].values

In [65]:
df_agg.tail()

absorbance0  absorbance1  absorbance2  ...  absorbance168  absorbance169  y
477     0.497722     0.502236     0.506524  ...       1.117265       1.120076  2
478     0.538159     0.543001     0.546593  ...       1.233912       1.243903  2
479     0.520465     0.523494     0.527131  ...       1.256473       1.263625  0
480     0.565816     0.570778     0.575131  ...       1.175161       1.174429  0
481     0.528848     0.528815     0.535568  ...       1.156578       1.137313  0

[5 rows x 171 columns]

In [66]:
X_columns=df_agg.columns[:-1]
y_columns=df_agg.columns[-1]

In [67]:
X_=df_agg[X_columns]
y=df_agg[y_columns]


In [68]:
X_

absorbance0  absorbance1  ...  absorbance168  absorbance169
0       0.505055     0.506570  ...       1.395545       1.369307
1       0.523531     0.523485  ...       1.302398       1.284716
2       0.516449     0.519226  ...       1.212862       1.193458
3       0.512397     0.514371  ...       1.266079       1.258333
4       0.496008     0.496698  ...       1.222712       1.213309
..           ...          ...  ...            ...            ...
477     0.497722     0.502236  ...       1.117265       1.120076
478     0.538159     0.543001  ...       1.233912       1.243903
479     0.520465     0.523494  ...       1.256473       1.263625
480     0.565816     0.570778  ...       1.175161       1.174429
481     0.528848     0.528815  ...       1.156578       1.137313

[482 rows x 170 columns]

In [69]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

__SCALER__="minmax"

if __SCALER__=="normal":
    normalizer = Normalizer() 
    X=normalizer.fit_transform(X_)
elif __SCALER__=="minmax":
    minmax = MinMaxScaler() 
    X=minmax.fit_transform(X_)


In [70]:
import tensorflow as tf
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout,Embedding, Flatten,LayerNormalization
from keras.models import Sequential, Model

import keras.optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld,mse,SparseCategoricalCrossentropy

from keras.preprocessing.sequence import pad_sequences

In [71]:
X

array([[0.4002052 , 0.3904527 , 0.40313755, ..., 0.76254299, 0.75596317,
        0.72258632],
       [0.480472  , 0.46664997, 0.4684968 , ..., 0.59226425, 0.57493468,
        0.56063031],
       [0.44970614, 0.44746225, 0.45665457, ..., 0.40972256, 0.40092392,
        0.3859072 ],
       ...,
       [0.46715235, 0.46668778, 0.47221709, ..., 0.52144124, 0.48568137,
        0.5202493 ],
       [0.66418082, 0.67968912, 0.68495695, ..., 0.36510925, 0.32765341,
        0.34947604],
       [0.50357307, 0.49065927, 0.50961407, ..., 0.32475253, 0.29153923,
        0.27841382]])

In [108]:
def blood_model():
    #clear_session()



    input_len=170

    print(input_len)
    output_size=24
    drop_frac0=0.0 
    drop_frac1=0.0



    input1=Input(shape=(input_len,),)

    #flatt=Flatten()(lstm1)

    non=42
    #initializer = tf.keras.initializers.LecunNormal()
    #initializer=tf.keras.initializers.LecunUniform()
    #initializer=tf.keras.initializers.HeUniform(    seed=None)
    #initializer= tf.keras.initializers.RandomNormal(    mean=3.0, stddev=0.05, seed=None)

    initializer="LecunUniform"
    d1=Dense(3000,activation="relu",kernel_initializer=initializer,input_dim=input_len)(input1)
    d1=Dropout(drop_frac0)(d1)
    d1=Dense(500,activation="relu",kernel_initializer=initializer,input_dim=input_len)(input1)
    d1=Dropout(drop_frac0)(d1)




    pred=Dense(3,activation="softmax")(d1)

    model = Model(inputs=input1, outputs=pred)

    opt = tf.keras.optimizers.Adam(learning_rate=0.001)


    lossfn = tf.keras.metrics.SparseCategoricalCrossentropy()

    model.compile(loss="SparseCategoricalCrossentropy",
        optimizer=opt,
        metrics=["SparseCategoricalCrossentropy","Accuracy"])
    return(model)

In [109]:
print(max(y))

2


In [110]:
new_model=blood_model()
!rm *.hdf5

170


In [111]:
!rm *.hdf5

rm: cannot remove '*.hdf5': No such file or directory


In [112]:
model_name="Newdata_col"
def scheduler(epoch, lr):
    return 0.000001

callback_LR = tf.keras.callbacks.LearningRateScheduler(scheduler)

callbacks = [callback_LR,
        
        #savemodela,
        ModelCheckpoint(filepath=model_name+"_{val_loss:5.5}.hdf5", monitor='val_loss',
                        verbose=1, save_best_only=True, mode='min')]

In [114]:
new_model.fit(X,
          y,
          epochs=1250, 
          batch_size=1,
          validation_split=0.1,
          verbose=1,
          callbacks=callbacks,
          shuffle=True
          
          )

Epoch 1/1250
426/433 [============================>.] - ETA: 0s - loss: 1.0541 - sparse_categorical_crossentropy: 1.0541 - Accuracy: 0.5117
Epoch 00001: val_loss did not improve from 1.04120
433/433 [==============================] - 1s 3ms/step - loss: 1.0533 - sparse_categorical_crossentropy: 1.0533 - Accuracy: 0.5127 - val_loss: 1.0507 - val_sparse_categorical_crossentropy: 1.0507 - val_Accuracy: 0.4694 - lr: 1.0000e-06
Epoch 2/1250
421/433 [============================>.] - ETA: 0s - loss: 1.0441 - sparse_categorical_crossentropy: 1.0441 - Accuracy: 0.5131
Epoch 00002: val_loss did not improve from 1.04120
433/433 [==============================] - 1s 3ms/step - loss: 1.0406 - sparse_categorical_crossentropy: 1.0406 - Accuracy: 0.5173 - val_loss: 1.0598 - val_sparse_categorical_crossentropy: 1.0598 - val_Accuracy: 0.4694 - lr: 1.0000e-06
Epoch 3/1250
422/433 [============================>.] - ETA: 0s - loss: 1.0302 - sparse_categorical_crossentropy: 1.0302 - Accuracy: 0.5213
Epoch 

KeyboardInterrupt: ignored

In [ ]:
X

In [ ]:
new_model.load_weights( "Newdata_col_0.00002_0.00003_.hdf5")

In [ ]:
new_model_bottle=Model(new_model.inputs,new_model.get_layer("bottle").output)
new_pred_bottle=new_model_bottle.predict(X)

In [ ]:
new_pred_bottle=new_model_bottle.predict(X)

In [ ]:
pca_df=pd.DataFrame(new_pred_bottle)
pca_df.columns=["0","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15"]

In [ ]:
pca_df["y"]=df_agg[y_columns]

In [ ]:
pca_df.head()

0         1         2         3  ...        13        14        15   y
0 -0.455455 -0.210233  4.744530 -0.163626  ...  4.544711  0.394087  4.037780  14
1 -0.487165 -0.134975  4.500106 -0.198809  ...  4.172063  0.649278  3.789781  12
2 -0.581075 -0.397844  4.716376 -0.013440  ...  4.410285  0.424075  3.868018  12
3 -0.427012 -0.037865  4.373947 -0.202987  ...  4.001109  0.803744  3.675647  12
4 -0.485447 -0.119334  4.234157 -0.105513  ...  3.930183  0.577152  3.545022  14

[5 rows x 17 columns]

In [ ]:
# x and y given as array_like objects
import plotly.express as px
fig = px.scatter(pca_df,x="3", y="4",color="y")
fig.show()

PCA dimenzió csökkentés

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca.fit(pca_df.iloc[:,:-1])


PCA(n_components=3)

In [ ]:
pca_df.iloc[:,:-1]

0         1         2  ...        13        14        15
0   -0.455455 -0.210233  4.744530  ...  4.544711  0.394087  4.037780
1   -0.487165 -0.134975  4.500106  ...  4.172063  0.649278  3.789781
2   -0.581075 -0.397844  4.716376  ...  4.410285  0.424075  3.868018
3   -0.427012 -0.037865  4.373947  ...  4.001109  0.803744  3.675647
4   -0.485447 -0.119334  4.234157  ...  3.930183  0.577152  3.545022
..        ...       ...       ...  ...       ...       ...       ...
477 -0.432547 -0.042612  4.151781  ...  3.745988  0.834010  3.430320
478 -0.545170 -0.269155  4.665953  ...  4.318055  0.604945  3.860862
479 -0.459478 -0.136953  4.505611  ...  4.148006  0.727984  3.771098
480 -0.578147 -0.267140  4.660335  ...  4.214275  0.782107  3.800989
481 -0.557550 -0.288060  4.563846  ...  4.157392  0.681090  3.740807

[482 rows x 16 columns]

In [ ]:
pca_out=pca.transform(pca_df.iloc[:,:-1])
pca_out_df=pd.DataFrame(pca_out)
pca_out_df["y"]=pca_df["y"]

In [ ]:
pca_out_df.head()

0         1         2   y
0 -0.937859 -0.175943 -0.025158  14
1 -0.097903  0.012702 -0.032255  12
2 -0.369259 -0.112322  0.410119  12
3  0.257648  0.112359 -0.128117  12
4  0.482845 -0.211820 -0.022614  14

In [ ]:
# x and y given as array_like objects
import plotly.express as px
fig = px.scatter(pca_out_df,x=0, y=1,color="y")
fig.show()

#Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score


clf = RandomForestClassifier(max_depth=8, random_state=0)
clf.fit(X, y)

randomforest_predict=clf.predict(X)

In [ ]:
print(f"Accuracy: {accuracy_score(y,randomforest_predict)}")

Accuracy: 0.8713692946058091


In [ ]:
y

0      14
1      12
2      12
3      12
4      14
       ..
477    20
478    21
479     4
480     6
481     4
Name: y, Length: 482, dtype: int64

In [ ]:
new_pred_bottle

array([[-0.4554547 , -0.21023284,  4.7445297 , ...,  4.544711  ,
         0.39408657,  4.03778   ],
       [-0.48716477, -0.13497454,  4.500106  , ...,  4.172063  ,
         0.64927834,  3.7897806 ],
       [-0.58107466, -0.39784446,  4.716376  , ...,  4.410285  ,
         0.42407537,  3.868018  ],
       ...,
       [-0.45947847, -0.13695267,  4.505611  , ...,  4.1480064 ,
         0.727984  ,  3.771098  ],
       [-0.5781468 , -0.26713985,  4.660335  , ...,  4.2142754 ,
         0.7821071 ,  3.8009887 ],
       [-0.5575501 , -0.28805977,  4.5638456 , ...,  4.157392  ,
         0.6810898 ,  3.7408068 ]], dtype=float32)

In [ ]:
def cosine_similarity(x, y):
    import numpy as np
    return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))

In [ ]:
# def a search matrix
def create_search_list(pred,df,str_index):
    out=[]
    for i,vec in enumerate(pred):
        out.append([vec,df[str_index].values[i]])
    return(out)
    
    

In [ ]:
__DELTA__=0.00005
def generate_similarity(it1,sim_table):
    out=[]
    for it2 in sim_table:
        
        diff=cosine_similarity(it1,it2[0])
        if 1-diff<__DELTA__:
            
            out.append(it2[1])
    return(out)

In [ ]:
def find_max(dict):
    max=0
    sum=0
    outkey=""
    for key in dict.keys():
        akt=dict[key]
        sum+=akt
        if max<akt:
            max=akt
            outkey=key
    return outkey,max

In [ ]:
def statistic(a):
    c=list(set(a))
    #print(c)
    out={}
    for i in c:
        out[i]=a.count(i)
    return(out)

In [ ]:
new_bottle_list=create_search_list(new_pred_bottle,df,"y")

In [ ]:
new_bottle_list

[[array([-0.4554547 , -0.21023284,  4.7445297 , -0.16362615,  4.947189  ,
         -0.30529305,  1.6940346 , -0.13908549, -0.40565306,  2.79469   ,
          3.9377806 , -0.75346386,  1.21402   ,  4.544711  ,  0.39408657,
          4.03778   ], dtype=float32), 12],
 [array([-0.48716477, -0.13497454,  4.500106  , -0.19880906,  4.4776974 ,
         -0.13144025,  1.6134118 , -0.10735638, -0.34769216,  2.6578715 ,
          3.5447726 , -0.68863255,  1.2659647 ,  4.172063  ,  0.64927834,
          3.7897806 ], dtype=float32), 12],
 [array([-0.58107466, -0.39784446,  4.716376  , -0.01344048,  4.5172863 ,
         -0.13337731,  1.6704023 , -0.10166623, -0.31629798,  2.7347178 ,
          3.4937572 , -0.70084304,  1.259092  ,  4.410285  ,  0.42407537,
          3.868018  ], dtype=float32), 12],
 [array([-0.42701238, -0.03786451,  4.373947  , -0.20298707,  4.3161645 ,
         -0.19946422,  1.5317276 , -0.08210754, -0.29096153,  2.5876327 ,
          3.4315352 , -0.6542869 ,  1.2998489 ,  4.001

In [ ]:
# submission generation
!rm "./Updated_Test.csv"

In [ ]:
!wget https://github.com/sipocz/Blood/raw/ae2dd49b40364eeb9e1821b70aa87ff332aca170/orig/Updated_Test.csv

--2021-12-11 12:01:53--  https://github.com/sipocz/Blood/raw/ae2dd49b40364eeb9e1821b70aa87ff332aca170/orig/Updated_Test.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Blood/ae2dd49b40364eeb9e1821b70aa87ff332aca170/orig/Updated_Test.csv [following]
--2021-12-11 12:01:54--  https://raw.githubusercontent.com/sipocz/Blood/ae2dd49b40364eeb9e1821b70aa87ff332aca170/orig/Updated_Test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10371075 (9.9M) [text/plain]
Saving to: ‘Updated_Test.csv’

Updated_Test.csv    100%[===================>]   9.89M  --.-KB/s    in 0.08s   

2021-12-11 12:01:5

In [ ]:
test_df=pd.read_csv("./Updated_Test.csv")
test_df.head()

test_df['hdl_cholesterol_human']=0
test_df["hemoglobin(hgb)_human"]=0
test_df["cholesterol_ldl_human"]=0

In [ ]:
test_df.head()

Reading_ID donation_id  ...  hemoglobin(hgb)_human  cholesterol_ldl_human
0  ID_KWUAQLRM     ID_2982  ...                      0                      0
1  ID_FN8SIVJJ     ID_2982  ...                      0                      0
2  ID_CI4TQCHI     ID_2982  ...                      0                      0
3  ID_URJMKNU7     ID_2982  ...                      0                      0
4  ID_HBJH67IS     ID_2982  ...                      0                      0

[5 rows x 179 columns]

In [ ]:
test_id_list_tmp=test_df.donation_id.values
test_id_list=[]
for id in test_id_list_tmp:
    if id not in test_id_list:
        test_id_list.append(id)


In [ ]:
cols=list(test_df.columns[2:172])

df_test_agg=pd.DataFrame()
for col in cols:
   
    result=test_df.groupby("donation_id").agg({col:func})
   
    df_test_agg[col]=result[col].iloc[:,0].values
df_test_agg["donation_id"]=result.index

In [ ]:
#result.index

In [ ]:
#df_test_agg.head()

In [ ]:
test_X=df_test_agg.iloc[:,0:170]

In [ ]:
#df_test_agg.describe()

In [ ]:
# calculate bottle vektor for test data
bottle_pred_test=new_model_bottle.predict(test_X)


In [ ]:
#new_bottle_list

In [ ]:
__DELTA__=0.00014
test_result=[]
db=0
for test_i in bottle_pred_test:
    o1=generate_similarity( test_i, new_bottle_list)
    print(statistic(o1), end="-->")
    result,_=find_max(statistic(o1))
    if result=="":
        result=12
        print("NINCS !!!")
    db+=1
    if (db % 50)==0:
        print("...")
    print(result,end=",")
    test_result.append(result)

{12: 3, 22: 1, 23: 3}-->12,{12: 9, 23: 4, 14: 4, 22: 8}-->12,{12: 11, 23: 7, 14: 5, 22: 5}-->12,{12: 4, 14: 1, 21: 1, 22: 5, 23: 4}-->22,{12: 1}-->12,{12: 4, 14: 2, 21: 1, 22: 5, 23: 4}-->22,{12: 8, 23: 8, 14: 5, 22: 15}-->22,{12: 12, 23: 3, 14: 3, 22: 3}-->12,{12: 9, 23: 4, 14: 3, 22: 7}-->12,{12: 13, 23: 5, 14: 2, 22: 10}-->12,{12: 3, 23: 1, 14: 2, 22: 3}-->12,{23: 3, 14: 1, 22: 2}-->23,{12: 12, 23: 11, 14: 2, 22: 7}-->12,{12: 2, 23: 2, 14: 1, 22: 5}-->22,{12: 2, 22: 2, 23: 6}-->23,{12: 4, 22: 1, 23: 1}-->12,{}-->NINCS !!!
12,{12: 3, 14: 3, 22: 3}-->12,{12: 13, 23: 7, 14: 3, 22: 10}-->12,{12: 17, 23: 17, 14: 8, 22: 21}-->22,{12: 14, 23: 2, 14: 3, 22: 7}-->12,{14: 1, 23: 2}-->23,{12: 5, 23: 4, 14: 5, 22: 4}-->12,{12: 4, 23: 3, 14: 2, 22: 7}-->22,{12: 17, 23: 11, 14: 3, 22: 8}-->12,{12: 20, 23: 9, 14: 8, 22: 19}-->12,{23: 4, 12: 4, 14: 1, 22: 4}-->23,{12: 6, 23: 5, 14: 1, 22: 7}-->22,{12: 15, 23: 15, 14: 4, 22: 15}-->12,{}-->NINCS !!!
12,{12: 6, 22: 4, 23: 3}-->12,{12: 1, 22: 3, 23: 1}

In [ ]:
test_result_randomforest=clf.predict(test_X)

In [ ]:
__OUT_GENERATOR__="Random Forest"

In [ ]:
if __OUT_GENERATOR__=="Neural":
    pred=_3_Label_encoder.inverse_transform(test_result)
elif __OUT_GENERATOR__=="Random Forest":
    pred=_3_Label_encoder.inverse_transform(test_result_randomforest)
    

In [ ]:
def de_conv(pred):
    out0=[]
    out1=[]
    out2=[]

    for p in pred:
        if p=="NaN":
            p="ok+ok+ok"
        print(p)
        
        out0.append(p.split("+")[0])
        out1.append(p.split("+")[1])
        out2.append(p.split("+")[2])
        
    return(out0,out1,out2)


In [ ]:
ret0,ret1,ret2=de_conv(pred)

In [ ]:
df_test_agg.head()

absorbance0  absorbance1  ...  absorbance169  donation_id
0     0.499920     0.504608  ...       1.027759      ID_1961
1     0.497758     0.498091  ...       1.227104      ID_2067
2     0.551436     0.553079  ...       1.433315      ID_2155
3     0.525926     0.528948  ...       1.192471      ID_2211
4     0.559814     0.559612  ...       1.295373      ID_2327

[5 rows x 171 columns]

In [ ]:
outdf=pd.DataFrame()
outdf["Reading_ID"]=df_test_agg["donation_id"]
outdf["donation_id"]=df_test_agg.donation_id
outdf["hdl_cholesterol_human"]=ret0
outdf["hemoglobin(hgb)_human"]=ret1
outdf["cholesterol_ldl_human"]=ret2

outdf.set_index("Reading_ID",inplace=True)

In [ ]:
outdf

donation_id  ... cholesterol_ldl_human
Reading_ID              ...                      
ID_1961        ID_1961  ...                  high
ID_2067        ID_2067  ...                    ok
ID_2155        ID_2155  ...                    ok
ID_2211        ID_2211  ...                    ok
ID_2327        ID_2327  ...                  high
...                ...  ...                   ...
ID_6373        ID_6373  ...                    ok
ID_6381        ID_6381  ...                    ok
ID_6391        ID_6391  ...                  high
ID_7755        ID_7755  ...                    ok
ID_7821        ID_7821  ...                  high

[61 rows x 4 columns]

In [ ]:
gen_outdf=pd.DataFrame()
c1=[]
c2=[]
for i in outdf.index:
    rec1=outdf.loc[i,"hdl_cholesterol_human"]
    rec2=outdf.loc[i,"hemoglobin(hgb)_human"]
    rec3=outdf.loc[i,"cholesterol_ldl_human"]
    c1.append(outdf.loc[i,"donation_id"]+"_"+"hdl_cholesterol_human")
    c2.append(rec1)
    c1.append(outdf.loc[i,"donation_id"]+"_"+"hemoglobin(hgb)_human")
    c2.append(rec2)
    c1.append(outdf.loc[i,"donation_id"]+"_"+"cholesterol_ldl_human")
    c2.append(rec3)
    
    

In [ ]:
print (c1)

['ID_1961_hdl_cholesterol_human', 'ID_1961_hemoglobin(hgb)_human', 'ID_1961_cholesterol_ldl_human', 'ID_2067_hdl_cholesterol_human', 'ID_2067_hemoglobin(hgb)_human', 'ID_2067_cholesterol_ldl_human', 'ID_2155_hdl_cholesterol_human', 'ID_2155_hemoglobin(hgb)_human', 'ID_2155_cholesterol_ldl_human', 'ID_2211_hdl_cholesterol_human', 'ID_2211_hemoglobin(hgb)_human', 'ID_2211_cholesterol_ldl_human', 'ID_2327_hdl_cholesterol_human', 'ID_2327_hemoglobin(hgb)_human', 'ID_2327_cholesterol_ldl_human', 'ID_2436_hdl_cholesterol_human', 'ID_2436_hemoglobin(hgb)_human', 'ID_2436_cholesterol_ldl_human', 'ID_2441_hdl_cholesterol_human', 'ID_2441_hemoglobin(hgb)_human', 'ID_2441_cholesterol_ldl_human', 'ID_2493_hdl_cholesterol_human', 'ID_2493_hemoglobin(hgb)_human', 'ID_2493_cholesterol_ldl_human', 'ID_2495_hdl_cholesterol_human', 'ID_2495_hemoglobin(hgb)_human', 'ID_2495_cholesterol_ldl_human', 'ID_2517_hdl_cholesterol_human', 'ID_2517_hemoglobin(hgb)_human', 'ID_2517_cholesterol_ldl_human', 'ID_2524_

In [ ]:
gen_outdf["Donation_ID"]=c1
gen_outdf["target"]=c2


In [ ]:
gen_outdf.head()

Donation_ID target
0  ID_1961_hdl_cholesterol_human     ok
1  ID_1961_hemoglobin(hgb)_human     ok
2  ID_1961_cholesterol_ldl_human   high
3  ID_2067_hdl_cholesterol_human   high
4  ID_2067_hemoglobin(hgb)_human     ok

In [ ]:
def create_submission_name(project):
    from datetime  import datetime
    strdate=datetime.now().strftime("%Y%m%d_%H%M%S")
    out=project+"_"+"submission"+"_"+strdate+".csv"
    return(out)

In [ ]:
gen_outdf.set_index("Donation_ID", inplace=True)

In [ ]:
submission_fname=create_submission_name("BloodAI")

In [ ]:
submission_fname

'BloodAI_submission_20211211_120156.csv'

In [ ]:
gen_outdf.to_csv(submission_fname,index=True)

In [ ]:
!head $submission_fname

Donation_ID,target
ID_1961_hdl_cholesterol_human,ok
ID_1961_hemoglobin(hgb)_human,ok
ID_1961_cholesterol_ldl_human,high
ID_2067_hdl_cholesterol_human,high
ID_2067_hemoglobin(hgb)_human,ok
ID_2067_cholesterol_ldl_human,ok
ID_2155_hdl_cholesterol_human,low
ID_2155_hemoglobin(hgb)_human,ok
ID_2155_cholesterol_ldl_human,ok
